In [1]:
from functools import singledispatch
from datetime import datetime

@singledispatch
def to_str(instance) -> str:
    """This is a definition to convert things to my string."""
    raise NotImplementedError

@to_str.register
def __to_str_with_number(instance: int | float | str) -> str:
    return str(instance)

@to_str.register 
def __to_str_with_list(instance: list) -> str:
    return ", ".join([to_str(x) for x in instance])

@to_str.register
def __to_str_with_datetime(instance: datetime):
    return instance.isoformat() 

to_str("12"), to_str([1, 2, 3]), to_str(datetime.now())


('12', '1, 2, 3', '2023-04-02T14:27:22.180261')

In [2]:
# abstract factory

from abc import ABC, abstractclassmethod


class Pet(ABC):
    
    def __init__(self, name: str) -> None:
        self.name = name
        
    @abstractclassmethod
    def speak(self):
        pass
    
    @abstractclassmethod
    def __str__(self):
        pass
    

class Dog(Pet):
    def speak(self):
        return 'Woof!'
    
    def __str__(self):
        return f'Dog {self.name}'


class Cat(Pet):
    def speak(self):
        return 'Meow!'

    def __str__(self):
        return f'Cat {self.name}'
    
    
class PetShop:
    def __init__(self, animal_factory: Pet | None) -> None:
        self.pet_factory = animal_factory
        
    def buy_pet(self, name: str) -> Pet:
        pet = self.pet_factory(name)
        print(f"this is your lovey {pet}")
        return pet
    

MyShop = PetShop(Dog)
pet = MyShop.buy_pet('Fido')
pet.speak()



this is your lovey Dog Fido


'Woof!'

In [9]:
# borg 
class Borg:
    _shared_state: dict[str, str] = {}
    
    def __init__(self) -> None:
        self.__dict__ = self._shared_state
        self.state = "Init"
        
    def __str__(self) -> str:
        return f"Current state : {self.state}"

        
class MyBorg(Borg):
    pass 


x = MyBorg()
y = MyBorg() 
print(x, "-", y)
y.state = "stopped"
print(x, "-", y)

Current state : Init - Current state : Init
Current state : stopped - Current state : stopped


In [11]:
# builder
class Building:
    def __init__(self) -> None:
        self.build_floor()
        self.build_size()

    def build_floor(self):
        raise NotImplementedError

    def build_size(self):
        raise NotImplementedError

    def __repr__(self) -> str:
        return "Floor: {0.floor} | Size: {0.size}".format(self)


# Concrete Buildings
class House(Building):
    def build_floor(self) -> None:
        self.floor = "One"

    def build_size(self) -> None:
        self.size = "Big"


class Flat(Building):
    def build_floor(self) -> None:
        self.floor = "More than One"

    def build_size(self) -> None:
        self.size = "Small"
        
def construct_building(cls) -> Building:
    building = cls()
    building.build_floor()
    building.build_size()
    return building

flat = construct_building(Flat)
house = construct_building(House)

flat, "-", house


(Floor: More than One | Size: Small, '-', Floor: One | Size: Big)

In [16]:
# If it walks like a duck and its quacks like a duck, then it must be a duck.

from dataclasses import dataclass
from typing import Protocol

class Item(Protocol):
    quantity: float
    price: float
    

@dataclass
class Product:
    product_name: str 
    quantity: float
    price: float 
    

@dataclass
class Stock:
    stock_name: str 
    quantity: float
    price: float 
    
def calculate_total_price(items: list[Item]) -> float:
    return sum([item.quantity * item.price for item in items])

total = calculate_total_price([Product("Apple", 10, 1.5), Product("Orange", 20, 2.0)])    
print(total)

total = calculate_total_price([Stock("Xiaomi", 5, 950), Stock("Mac", 20, 1000)])
print(total)

55.0
24750


In [19]:
# factory
from dataclasses import dataclass
from typing import Type, Protocol


class Localizer(Protocol):
    
    def localize(self, message: str) -> str:
        pass

@dataclass
class VietnamLocalizer:
    translations = {
        "hello": "xin chào",
        "everyone": "mọi người",
    }
    
    def localize(self, message: str) -> str:
        return self.translations.get(message, message)
    
@dataclass
class EnglishLocalizer:
    
    def localize(self, message: str) -> str:
        return message

def get_localizer(language: str = "English") -> Localizer:
    
    localizers: dict[str, Type[Localizer]] = {
        "English": EnglishLocalizer,
        "Vietnam": VietnamLocalizer,
    }
    
    return localizers[language]()

e, v = get_localizer("English"), get_localizer("Vietnam")

for msg in "hello everyone".split():
    print(e.localize(msg), "-->", v.localize(msg))

hello --> xin chào
everyone --> mọi người


In [2]:
# prototype

class Prototype:
     
    def __init__(self, value: str = "Default", **args) -> None:
        self.value = value
        self.__dict__.update(args)
        
    def clone(self, **args) -> "Prototype":
        obj = self.__class__(**self.__dict__)
        obj.__dict__.update(args)

        return obj
    
class PrototypeDispatcher:
    
    def __init__(self):
        self._objects = {}
        
    def get_objects(self)-> dict[str, Prototype]:
        return self._objects
    
    def register_object(self, name: str, obj: Prototype) -> None:
        self._objects[name] = obj
        
    def unregister_object(self, name: str) -> None:
        self._objects.pop(name)
        

dispatcher = PrototypeDispatcher()
prototype = Prototype()

x = prototype.clone()
y = prototype.clone(value="OK", type="str")
z = prototype.clone(value=1, is_checked=True)

dispatcher.register_object("x", x)
dispatcher.register_object("y", y)
dispatcher.register_object("z", z)

print([{n: p.value} for n, p in dispatcher.get_objects().items()])


[{'x': 'Default'}, {'y': 'OK'}, {'z': 1}]


In [16]:
from typing import Callable, TypeVar

class Dog:
    def __init__(self) -> None:
        self.name = "Dog"

    def bark(self) -> str:
        return "woof!"


class Cat:
    def __init__(self) -> None:
        self.name = "Cat"

    def meow(self) -> str:
        return "meow!"
    
class Human:
    def __init__(self) -> None:
        self.name = "Human"

    def speak(self) -> str:
        return "hello"


class Car:
    def __init__(self) -> None:
        self.name = "Car"

    def make_noise(self, octane_level: int) -> str:
        return f"brum brum {'!' * octane_level}"
    

T = TypeVar("T")

class Adapter:
    def __init__(self, object: T, **adapter_method: Callable) -> None:
        self.object = object
        self.__dict__.update(adapter_method)
        
    def __getattr__(self, attr: str) -> Callable:
        return getattr(self.object, attr)
    
    def orginal_dict(self) -> dict:
        return self.object.__dict__
    

objects = [] 

cat = Cat()
dog = Dog()
car = Car()
human = Human()

objects.append(Adapter(cat, make_noise=cat.meow))
objects.append(Adapter(dog, make_noise=dog.bark))
objects.append(Adapter(car, make_noise=lambda: car.make_noise(3)))
objects.append(Adapter(human, make_noise=human.speak))

for obj in objects:
    print(f"A {obj.name} goes {obj.make_noise()}")

A Cat goes meow!
A Dog goes woof!
A Car goes brum brum !!!
A Human goes hello


In [17]:
class CPU:
    """
    Simple CPU representation.
    """

    def freeze(self) -> None:
        print("Freezing processor.")

    def jump(self, position: str) -> None:
        print("Jumping to:", position)

    def execute(self) -> None:
        print("Executing.")


class Memory:
    """
    Simple memory representation.
    """

    def load(self, position: str, data: str) -> None:
        print(f"Loading from {position} data: '{data}'.")


class SolidStateDrive:
    """
    Simple solid state drive representation.
    """

    def read(self, lba: str, size: str) -> str:
        return f"Some data from sector {lba} with size {size}"


class ComputerFacade:
    """
    Represents a facade for various computer parts.
    """

    def __init__(self):
        self.cpu = CPU()
        self.memory = Memory()
        self.ssd = SolidStateDrive()

    def start(self):
        self.cpu.freeze()
        self.memory.load("0x00", self.ssd.read("100", "1024"))
        self.cpu.jump("0x00")
        self.cpu.execute()



computer_facade = ComputerFacade()
computer_facade.start()

Freezing processor.
Loading from 0x00 data: 'Some data from sector 100 with size 1024'.
Jumping to: 0x00
Executing.


In [56]:
# timer decorator
from time import perf_counter
from datetime import datetime
from functools import wraps

def timer(func):
    """this decorator prints out the execution time of a callable"""
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        print(f"{func.__name__} took {end - start} seconds to execute.")
        return result
    
    return wrapper


def logexc(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        print(kwargs)
        sig = ", ".join(args_repr + kwargs_repr)
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print("Time: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            print("Arguments:", sig)
            print("Error: ", e)
        
    return wrapper
        
        
@timer 
@logexc
def dothings(n_times):
    for _ in range(n_times):
        return sum([i**2 for i in range(10000)])
    
dothings(1000)

{}
Time:  2023-04-02 20:54:56
Arguments: 1000
Error:  'str' object cannot be interpreted as an integer
dothings took 0.00013481800033332547 seconds to execute.


In [64]:
from functools import partial
import requests 

def retry(func=None, n_tries: int = 3):
    
    if not func:
        return partial(retry, n_tries=n_tries)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        tries = 0 
        while True:
            resp = func(*args, **kwargs)
            if resp.status_code != 200 and tries < n_tries:
                print("Retrying... ", tries)
                tries += 1
                continue 
            break 
        return resp 
    return wrapper 


# ver2 - create outer function for retry decorator
def retry2(n_tries: int = 3):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            tries = 0 
            while True:
                resp = func(*args, **kwargs)
                if resp.status_code != 200 and tries < n_tries:
                    print("Retrying... ", tries)
                    tries += 1
                    continue 
                break 
            return resp 
        return wrapper 
    return decorator

@retry(n_tries=3)
# @retry2(n_tries=3)
def crawl(url: str):
    return requests.get(url)

crawl("https://www.tfs.com")


Retrying...  0
Retrying...  1
Retrying...  2


<Response [403]>

In [66]:
# try decorator with class 
from functools import update_wrapper

class Tally:
    def __init__(self, func):
        update_wrapper(self, func)
        self.func = func
        self.tally = {}
        self.n_calls = 0 
        
    def __call__(self, *args, **kwargs):
        self.n_calls += 1
        self.tally[self.func.__name__] = self.n_calls
        
        print(f"Function {self.func.__name__} has been called {self.n_calls} times.")
        return self.func(*args, **kwargs)
    
@Tally 
def foo(name: str):
    return f"Hello {name}"

foo("John"), foo("haucx")

Function foo has been called 1 times.
Function foo has been called 2 times.


('Hello John', 'Hello haucx')

In [73]:
from enum import Enum 

class ConvertMetersTo(Enum):
    FEET = 3.28084
    YARDS = 1.09361
    MILES = 0.000621371
    
    def convert(self, value: float) -> float:
        return value * self.value
    
def convert(func=None, convert_to=None):
    if func is None:
        return partial(convert, convert_to=convert_to)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Conversion to: ", convert_to)
        
        ret = func(*args, **kwargs)
        
        if convert_to is None:
            return ret
        elif convert_to in ConvertMetersTo.__members__:
            return ConvertMetersTo[convert_to].convert(ret)
        else:
            raise ValueError(f"Conversion to {convert_to} is not supported.")
        
    return wrapper

@convert(convert_to="MILES")
def area(a, b):
    return a*b 

result = area(10, 20)
print(f"after connvert {10*20} meters to miles: ", result)
        

Conversion to:  MILES
after connvert 200 meters to miles:  0.1242742


In [4]:
x = {'spam': 1, 'eggs': 2, 'cheese': 3}
y = {'cheese': 5, 'extra': 4}


x | y

{'spam': 1, 'eggs': 2, 'cheese': 5, 'extra': 4}

In [13]:
import time


def timed(f):
    def timed_f(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(elapsed)
        return value

    return timed_f


@timed
def expensive_f():
    return sum(i ** 2 for i in range(1000000))


# expensive_f = timed(expensive_f)

class Button:
    def on_clicked(self):
        print('You clicked me')

    def register_on_clicked(self, f):
        self.on_clicked = f
        return f


buttons = [Button() for _ in range(10)]


@buttons[3].register_on_clicked
def print_hello():
    print('hello')


if __name__ == '__main__':
    print(expensive_f())

    for button in buttons:
        button.on_clicked()

0.1048231049990136
333332833333500000
You clicked me
You clicked me
You clicked me
hello
You clicked me
You clicked me
You clicked me
You clicked me
You clicked me
You clicked me
